In [476]:
source("functions_working.R")
require(ggplot2)
require('dplyr')


 loading required packages



In [487]:
trees_cen        = read.csv("../Results/trees_census_dates.csv")
trees_df         = read.csv("../Results/trees_matrix.csv", row.names=1)
pca.rslt_trees   = do_pca(trees_df, scale = F, plot = F)

#cat("Explained Varience Trees:")
#pca.rslt_trees@exp.var

In [543]:
btles_cen        = read.csv("../Results/beetles_census_dates.csv")
btles_df         = read.csv("../Results/beetles_matrix.csv", row.names=1)
pca.rslt_btles   = do_pca(btles_df, scale = F, plot = F)

In [565]:
standardise_time = function(df, axis, census_time){
    
    df = df[, axis]
    
    plts = unique(unlist(strsplit(rownames(df), "_"))[c(T, F, F)])
    splt = unique(unlist(strsplit(rownames(df), "_"))[c(F, T, F)])
    cens = unique(unlist(strsplit(rownames(df), "_"))[c(F, F, T)])
    
    raw = array(dim= c(length(splt), ncol(df), length(cens), length(plts)),
                dimnames = list(splt, axis, cens, plts))
    
    adj = raw
    
    for (plt in plts){ for (sp in splt){ for (cen in cens){
        raw[sp, , cen, plt] = unlist(df[paste(plt, sp, cen, sep = "_"), ])
    }}}
    
    for (plt in plts){ for (pc in 1:3){ 
        tmp = raw[,pc,,plt]
        tmp = tmp[, !apply(is.na(tmp), 2, all)]
        tmp[which(rowSums(is.na(tmp)) > 0), ] = NA
    
        for (c in colnames(tmp)){
            raw[,pc,c,plt] = tmp[,c]
        }
    }}
    
    adj[,,cens[1],] = raw[,,cens[1],]
    
    for (plt in plts){ for (c in 2:length(cens)){
    mask = census_time$plot == plt & census_time$census == cens[c]
    if (sum(mask) == 0) {
        adj[ , , cens[c], plt] = NA
        } else {
        diff_yrs = census_time[mask, "diff_yrs"]
    
        adj[,,cens[c], plt] = raw[,,cens[c-1], plt] +
                              ((raw[,,cens[c], plt] - raw[,,cens[c-1], plt])*(1/diff_yrs)) 
        }
    }}
    
    adj = adply(adj, c(4, 1, 3))
    colnames(adj)[1:3] = c('plot','subplot', 'census')
    rownames(adj) = paste(adj$plot, adj$subplot, adj$census, sep = "_")

    adj = adj[order(rownames(adj)), ]
    
    adj = adj[, axis]
    
    return(adj)
}

In [569]:
a = standardise_time(pca.rslt_btles@axis, c("PC1", "PC2", "PC3"), btles_cen)

In [583]:
a["VJR_579_P3",]

,PC1,PC2,PC3
VJR_579_P3,NA,NA,NA


In [579]:
pca.rslt_btles@axis[, c("PC1", "PC2", "PC3")]

,PC1,PC2,PC3
A_10_P1,-0.008952667,-0.007049194,0.0029139363
A_10_P2,-0.013097520,-0.011299292,0.0004784716
A_10_P3,-0.011676032,0.020009982,0.0101834622
A_11_P2,-0.003437009,-0.011282324,-0.0004279692
A_11_P3,-0.007585163,0.001781069,0.0083506385
A_12_P2,-0.004817059,-0.011286959,-0.0003022451
A_12_P3,0.008951935,-0.002239424,0.0022978896
A_13_P2,-0.003438699,-0.011201791,-0.0002997509
A_13_P3,0.008981600,-0.006247371,-0.0024262306
A_14_P2,-0.003437009,-0.011282324,-0.0004279692


In [581]:
b["VJR_579_P3", ]

,PC1,PC2,PC3
VJR_579_P3,0.002102239,0.005687852,0.01046642


In [548]:
pca  = pca.rslt_btles@axis[, c("PC1", "PC2", "PC3")]

plts = unique(unlist(strsplit(rownames(pca), "_"))[c(T, F, F)])
splt = unique(unlist(strsplit(rownames(pca), "_"))[c(F, T, F)])
cens = unique(unlist(strsplit(rownames(pca), "_"))[c(F, F, T)])


In [549]:
test = array(dim= c(length(splt), ncol(pca), length(cens), length(plts)),
             dimnames = list(splt, colnames(a), cens, plts))

test2 = test

for (plt in plts){ for (sp in splt){ for (cen in cens){
  test[sp, , cen, plt] = unlist(a[paste0(plt, "_", sp, "_", cen), ]) }}}

In [550]:
for (plt in plts){ for (pc in 1:3){ 
    tmp = test[,pc,,plt]
    tmp = tmp[, !apply(is.na(tmp), 2, all)]
    tmp[which(rowSums(is.na(tmp)) > 0), ] = NA
    
    for (c in colnames(tmp)){
    test[,pc,c,plt] = tmp[,c]
    }
}}

In [552]:
test2[,,cens[1],] = test[,,cens[1],]

In [558]:
for (plt in plts){ for (c in 2:4){
    mask = census_time$plot == plt & census_time$census == cens[c]
    if (sum(mask) == 0) {
        test2[ , , cens[c], plt] = NA
        } else {
        diff_yrs = trees_cen[mask, "diff_yrs"]
    
        test2[,,cens[c], plt] = test[,,cens[c-1], plt] +
                                ((test[,,cens[c], plt] - test[,,cens[c-1], plt])*(1/diff_yrs)) 
        }
    }}

ERROR: Error in if (sum(mask) == 0) {: missing value where TRUE/FALSE needed


In [564]:
cens

[1] "P1" "P2" "P3"

In [527]:
test2 = adply(test2, c(4, 1, 3))

In [533]:
test2 = adply(test2, c(4, 1, 3))
colnames(test2)[1:3] = c('plot','subplot', 'census')
rownames(test2) = paste(test2$plot, test2$subplot, test2$census, sep = "_")

test2 = test2[order(rownames(test2)), ]

In [ ]:
################################# hvs_rslts ###################################

#' function produces and compares hypervolumes
#' 
#' returns a object of class hv.rslts with summary stats and comparisons of hypervolumes
#' @param df a dataframe which is the axis of hypervolume as columns and plots/census as rows 
#' @param axis - vector of axis from which to draw hypervolumes
#' @export
#' @examples
#' hvs_rslts(df, axis = c("PC1", "PC2", "PC3"))

hvs_rslts <- function(df, axis, what = "seq"){

  df <- scale_axis(df, axis)

  # make list of hv names to work off 
  names <- c()
  for (p in unique(df$plot)){ for (c in unique(df$census)){
    names <- c(names, paste0(p, "_", c)) }}

  cols = c("plot", "census", paste0("centroid_", axis), paste0(axis, "_l"),
           paste0(axis, "_h"))

  # empty list to store hypervolumes as they're built
  hvlist <- list()

  # construct df to store summary stats for each hv
  rslts           <- data.frame(matrix(NA, nrow = length(names), ncol = length(cols)))
  colnames(rslts) <- cols
  rownames(rslts) <- names
  rslts$plot_c    <- names

  cat("\n")
    
  for (i in names){
    
    cat("\rBuilding Hypervolume ", sprintf("%02d", which(names == i)), " of ",
        sprintf("%02d", length(names)), ": ",
        sprintf(paste0("%-0", max(nchar(names)), "s"), i))

    p <- unlist(strsplit(i, "_"))[[1]]  # plot
    c <- unlist(strsplit(i, "_"))[[2]]  # census

    # subset of the pca space with require plot/census and PCs
    tmp <- subset(df, plot == p & census == c,
                  select = colnames(df)[!(colnames(df) %in% c("plot", "subplot", "census"))])

    if (nrow(tmp) < 2){
      hv <- NA
    } else{
      hv <- hypervolume_gaussian(tmp, name = i, verbose = FALSE)
    }
    
    hvlist[[i]] <- hv
      
    rslts[i, "plot"]   <- unlist(strsplit(i, "_"))[1]
    rslts[i, "census"] <- unlist(strsplit(i, "_"))[2]

    if (class(hv) == "Hypervolume"){

      rslts[i, "volume"]       <- hv@Volume

      a <- as.data.frame(get_centroid(hv))

      for (z in rownames(a)){
        rslts[i, paste0("centroid_", z)] <- a[z, ]
        rslts[i, paste0(z, "_l")]        <- min(hv@RandomPoints[,z])
        rslts[i, paste0(z, "_h")]        <- max(hv@RandomPoints[,z])
      }
    }     
  }
  
  # put results through compare census
  compare <- compare_census(rslts, hvlist, what = what)

  # save output to hv.rslts class
  out <- new("hv.rslts", hvlist = hvlist, rslts = rslts, compare = compare)

  cat("\n")

  return(out)
}

In [8]:
hvs_rslts_trees   = hvs_rslts(pca.rslt_trees@axis,   axis = c("PC1", "PC2", "PC3"), "seq")


Building Hypervolume  36  of  36 :  Tower_c4 
Comparing Hypervolume 27  of  27 :  Tower_c3-c4 


In [2]:
litter_df = read.csv("../Data/RAW/playing/Litterfall.csv")
finert_df = read.csv("../Data/RAW/playing/FineRoots.csv")

In [3]:
litter_df$dates   = as.Date(litter_df$Collection.date, format = "%d/%m/%Y")
litter_df$plot    = gsub(" ", "", litter_df$Plot)
litter_df$subplot = litter_df$Trap

In [4]:
# DEALING WITH THE DUPLICATE PLOT!?
litter_df <- litter_df[!duplicated(litter_df[,c("dates", "plot", "subplot")]), ]

In [5]:
plts = as.character(unique(litter_df$plot))

In [6]:
census_df = data.frame(dates = NA, census = NA, plot = NA)

for (i in plts){
    tmp    = subset(litter_df, plot == i)
    dates  = sort(unique(tmp$dates))
    census =  paste0("c", 1:length(dates))
    tmp = data.frame(dates, census, plot = i)
    census_df = rbind(tmp, census_df)
}

census_df = census_df[-nrow(census_df),]

litter_df = merge(litter_df, census_df, by = c("dates", "plot"))

In [7]:
rownames(litter_df) = paste0(litter_df$plot, "_", litter_df$subplot, "_", litter_df$census)

In [8]:
hv_df = litter_df[, c("Leaves..Mg.C.ha.1.year.1", "Twigs..Mg.C.ha.1.year.1",
                      "Fruit..Mg.C.ha.1.year.1", "Flower..Mg.C.ha.1.year.1", "Seed..Mg.C.ha.1.year.1",
                     "Miscellaneous..Mg.C.ha.1.year.1", "plot", "subplot", "census")]
colnames(hv_df) = c("leaves", "twigs", "fruit", "flower", "seed", "misc", "plot", "subplot", "census")

hv_df$leaves = as.numeric(levels(hv_df$leaves)[hv_df$leaves])
hv_df$twigs = as.numeric(levels(hv_df$twigs)[hv_df$twigs])
hv_df$fruit = as.numeric(levels(hv_df$fruit)[hv_df$fruit])
hv_df$flower = as.numeric(levels(hv_df$flower)[hv_df$flower])
hv_df$seed = as.numeric(levels(hv_df$seed)[hv_df$seed])
hv_df$misc = as.numeric(levels(hv_df$misc)[hv_df$misc])
hv_df[is.na(hv_df)] = 0

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [9]:
hvs_rslts(hv_df, axis = c("leaves", "twigs", "misc"))


Building Hypervolume  14  of  583 :  BNorth_c14

ERROR: Error in hypervolume_gaussian(tmp, name = i, verbose = FALSE): Bandwidth must be non-zero.


In [189]:
df = scale_axis(hv_df, axis = c("leaves", "twigs", "misc"))

In [191]:
subset(hv_df, plot == "BNorth" & census == "c14")

,leaves,twigs,fruit,flower,seed,misc,plot,subplot,census
BNorth_1_c14,3.0736842,0.3842105,0,0.0000000,0,0,BNorth,1,c14
BNorth_2_c14,1.8442105,0.8836842,0,0.0000000,0,0,BNorth,2,c14
BNorth_3_c14,0.6147368,0.0000000,0,0.3842105,0,0,BNorth,3,c14
BNorth_4_c14,4.2263158,0.5378947,0,0.0000000,0,0,BNorth,4,c14
BNorth_5_c14,0.3842105,0.0000000,0,0.0000000,0,0,BNorth,5,c14
BNorth_6_c14,3.2657895,0.0000000,0,0.0000000,0,0,BNorth,6,c14
BNorth_7_c14,2.7278947,0.6915789,0,1.2678947,0,0,BNorth,7,c14
BNorth_8_c14,1.7673684,0.0000000,0,0.0000000,0,0,BNorth,8,c14
BNorth_9_c14,2.6126316,0.1921053,0,0.0000000,0,0,BNorth,9,c14
BNorth_10_c14,3.4963158,2.9584211,0,0.0000000,0,0,BNorth,10,c14


## Other measures of stability

In [1]:
trees_df   = read.csv("../Results/trees_genus_matrix.csv", row.names=1)
mammals_df = read.csv("../Results/mammals_matrix.csv", row.names=1)
beetles_df = read.csv("../Results/beetles_matrix.csv", row.names=1)

In [2]:
trees_df

,Actinodaphne,Adenanthera,Adinandra,Aglaia,Aidia,Alangium,Alphonsea,Alseodaphne,Alstonia,Anisophyllea,⋯,Vatica,Vitex,Walsura,Xanthophyllum,Xanthopyllum,Xylopia,Ziziphus,indet,orophea,scaphium
BNorth_sp10_c1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,2,0,0,0,0
BNorth_sp10_c2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,2,0,0,0,0
BNorth_sp10_c3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,3,0,0,0,0
BNorth_sp10_c4,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,2,0,0,0,0
BNorth_sp11_c2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
BNorth_sp11_c3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
BNorth_sp12_c1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
BNorth_sp12_c2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
BNorth_sp12_c3,0,0,0,1,0,0,0,0,0,0,⋯,0,1,0,0,0,1,0,0,0,0
BNorth_sp12_c4,0,0,0,1,0,0,0,0,0,0,⋯,0,0,0,0,0,1,0,0,0,0
